In [8]:
import numpy as np
    
def generateVtwinsBA(vol_params = None,psf_params = None): 
    # Uout2 = generateVtwinsBA(vol_params,psf_params)
    
    # This function generates a Vtwins back aperture intensity profile with
# the specified aberrations applied
    
    
    # - vol_params          - Struct with parameters for the volume generation
#          .vol_sz      - 3-element vector with the size (in um) of the
#                         volume to generate (default = 100x100x30um)
#          .vres        - resolution to simulate volume at (default = 2
#                         samples/um)
# - psf_params          - Struct contaning the parameters for the PSF
#          .NA          - Numerical aperture of Gaussian beam
#          .n           - Refractive index of propagation material
#          .lambda      - Two-photon excitation wavelength (um)
#          .obj_fl      - Objective focal length (mm)
#          .ss          - Subsampling factor for fresnel propagation
#          .sampling    - Spatial sampling for tissue occlusion mask
#          .zernikeDst  - Microscope aberration weights (Zernike basis) as
#                         a function of position
    
    # - Uout2               - Output scalar field
    
    # 2017 - Alex Song
    
    ###########################################################################
    
    fl = np.float32(psf_params.obj_fl / 1000)
    
    D2 = np.float32(1e-06 * (1 / vol_params.vres) / psf_params.ss)
    
    N = np.float32(1e-06 * (vol_params.vasc_sz(np.arange(1,2+1)) - vol_params.vol_sz(np.arange(1,2+1))) / D2)
    
    D1 = np.float32(np.amax(gaussianBeamSize(psf_params,fl * 1000000.0) / 1000000.0) / np.amin(N))
    
    nre = np.float32(psf_params.n)
    
    rad = np.float32(np.tan(np.arcsin(psf_params.objNA / nre)) * fl)
    
    k = 2 * nre * np.pi / np.float32(psf_params.vlambda * 1e-06)
    
    X,Y = np.meshgrid((np.arange(- N(1) / 2,N(1) / 2 - 1+1)) * D1,(np.arange(- N(2) / 2,N(2) / 2 - 1+1)) * D1)
    beamRad = np.float32(np.tan(np.arcsin(psf_params.beamNA / nre)) * fl)
    
    beamOffset = np.float32(tand(psf_params.theta / 2)) * fl
    
    if (not isfield(psf_params,'vtwinstype')  or len(psf_params.vtwinstype)==0):
        UoutL = generateGaussianProfile(X,Y,beamRad,rad,k,fl,np.array([beamOffset,0]))
        UoutR = generateGaussianProfile(X,Y,beamRad,rad,k,fl,np.array([- beamOffset,0]))
    else:
        if (str(psf_params.vtwinstype) == str('bessel')):
            width = 1.4e-10 * np.tan(np.arcsin(psf_params.NA / nre)) / (fl * 1e-06 * psf_params.length)
            if (not isfield(psf_params,'bprof')  or len(psf_params.bprof)==0):
                UoutL = generateBesselProfile(X,Y,beamRad,width,k,fl,'gaussian',rad,np.array([beamOffset,0]))
                UoutR = generateBesselProfile(X,Y,beamRad,width,k,fl,'gaussian',rad,np.array([- beamOffset,0]))
            else:
                UoutL = generateBesselProfile(X,Y,beamRad,width,k,fl,psf_params.bprof,inf,np.array([beamOffset,0]))
                UoutR = generateBesselProfile(X,Y,beamRad,width,k,fl,psf_params.bprof,inf,np.array([- beamOffset,0]))
    
    tilt = np.array([0,1,0]) * psf_params.vlambda * 1e-06 * psf_params.sepdist / 8
    UoutL = applyZernike(UoutL,X / rad,Y / rad,k,tilt)
    UoutR = applyZernike(UoutR,X / rad,Y / rad,k,- tilt)
    imax = np.round(vol_params.vol_sz(1) / psf_params.sampling) + 1
    jmax = np.round(vol_params.vol_sz(2) / psf_params.sampling) + 1
    if (imax * jmax == 1 or (not isfield(psf_params,'zernikeDst') ) or len(psf_params.zernikeDst)==0):
        abb = generateZernike(psf_params)
        Uout2L = applyZernike(UoutL,X / rad,Y / rad,k,abb)
        Uout2R = applyZernike(UoutR,X / rad,Y / rad,k,abb)
    else:
        Uout2L = cell(imax,jmax)
        Uout2R = cell(imax,jmax)
        for i in np.arange(1,imax+1).reshape(-1):
            for j in np.arange(1,jmax+1).reshape(-1):
                abb = generateZernike(psf_params)
                Uout2L[i,j] = applyZernike(Uout2L,X,Y,k,abb)
                Uout2R[i,j] = applyZernike(Uout2R,X,Y,k,abb)
    
    Uout2.left = Uout2L
    Uout2.right = Uout2R
    return Uout2